In [1]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import jax

# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)

jax.config.update('jax_log_compiles', False)
jax.config.update('jax_check_tracer_leaks', False)

In [2]:
import jax.numpy as jnp
a = jnp.array(range(1000)) 
a.sum()

DeviceArray(499500, dtype=int32)

In [3]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib
from mimicnet import concept
from mimicnet import jax_interface
from mimicnet import dag
from mimicnet import glove
from mimicnet import gram
from mimicnet import ode
from mimicnet import models

importlib.reload(sys.modules['mimicnet.concept'])
importlib.reload(sys.modules['mimicnet.dag'])
importlib.reload(sys.modules['mimicnet.jax_interface'])
importlib.reload(sys.modules['mimicnet.glove'])
importlib.reload(sys.modules['mimicnet.gram'])
importlib.reload(sys.modules['mimicnet.ode'])
importlib.reload(sys.modules['mimicnet.models'])

/home/asem/.conda/envs/mimic3-snonet/lib/python3.9/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '


<module 'mimicnet.models' from '/home/asem/GP/MIMIC-SNONET/mimicnet/models.py'>

In [4]:

KG = dag.CCSDAG()

In [5]:
# multi_visit_mimic_dir = '/home/am8520/GP/ehr-data/mimic3-multi-visit'
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic3-multi-visit'
transformed_mimic_dir = '/home/asem/GP/ehr-data/mimic3-transforms'
mimic_dir = '/home/asem/GP/ehr-data/mimic3-v1.4/physionet.org/files/mimiciii/1.4'
# mimic_dir = '/home/asem/GP/MIMIC-SNONET/RAW/mimic-iii-clinical-database-1.4'

experiments_dir = '/home/asem/GP/ehr-data/mimic3-snonet-exp'
experiment_prefix = 'DEC03'

In [6]:
D_LABITEMS = pd.read_csv(f'{mimic_dir}/D_LABITEMS.csv.gz')
D_ITEMS = pd.read_csv(f'{mimic_dir}/D_ITEMS.csv.gz')
D_TEST = pd.concat([D_LABITEMS, D_ITEMS], join='inner')
test_label_dict = dict(zip(D_TEST.ITEMID, D_TEST.LABEL))
test_cat_dict = dict(zip(D_TEST.ITEMID, D_TEST.CATEGORY))

In [7]:
static_df = pd.read_csv(f'{transformed_mimic_dir}/static_df.csv.gz')
adm_df = pd.read_csv(f'{transformed_mimic_dir}/adm_df.csv.gz')
diag_df = pd.read_csv(f'{transformed_mimic_dir}/diag_df.csv.gz', dtype={'ICD9_CODE': str})
proc_df = pd.read_csv(f'{transformed_mimic_dir}/proc_df.csv.gz', dtype={'ICD9_CODE': str})
test_df = pd.read_csv(f'{transformed_mimic_dir}/test_df.csv.gz')


In [8]:
# Cast columns of dates to datetime64

static_df['DOB'] = pd.to_datetime(static_df.DOB, infer_datetime_format=True).dt.normalize()
adm_df['ADMITTIME'] = pd.to_datetime(adm_df.ADMITTIME, infer_datetime_format=True).dt.normalize()
adm_df['DISCHTIME'] = pd.to_datetime(adm_df.DISCHTIME, infer_datetime_format=True).dt.normalize()
test_df['DATE'] = pd.to_datetime(test_df.DATE, infer_datetime_format=True).dt.normalize()

In [9]:
static_df.ETHNIC_GROUP.nunique()

7

In [10]:
test_df

,SUBJECT_ID,ITEMID,DATE,VALUE
0,17,50852,2134-12-22,-1.201339
1,17,50861,2134-12-22,-0.809370
2,17,50862,2134-12-22,1.634410
3,17,50863,2134-12-22,-1.452596
4,17,50867,2134-12-22,-0.608894
...,...,...,...,...
4546204,99982,227456,2157-02-22,1.794272
4546205,99982,227457,2157-02-22,-0.723781
4546206,99982,227465,2157-02-22,2.247434
4546207,99982,227466,2157-02-22,-0.029395


In [11]:
patients = concept.Subject.to_list(static_df, adm_df, diag_df, proc_df, test_df)

In [12]:
len(patients)

4434

In [13]:
# ehr_dfs = concept.Subject.to_df(patients)

In [14]:
# def df_eq(df1, df2):
#     df11 = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
#     df22 = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
#     return df11[df22.columns].equals(df22)

# all(map(df_eq, ehr_dfs, [static_df, adm_df, diag_df, proc_df, test_df]))

### [FORK] Skip the cell below to load the jaxified data from a stored file on disc

In [15]:
# subjects_interface = jax_interface.SubjectJAXInterface(patients, set(test_df.ITEMID), KG)
# import pickle
# with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'wb') as pickleFile:
#     pickle.dump(subjects_interface, pickleFile)

In [16]:
import pickle
with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'rb') as pickleFile:
    subjects_interface = pickle.load(pickleFile)

## GloVe Initialization

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

In [19]:
glove_args = {
    'diag_idx': subjects_interface.diag_multi_ccs_idx,
    'proc_idx': subjects_interface.proc_multi_ccs_idx,
    'ccs_dag': KG,
    'subjects': patients,
    'diag_vector_size': 100,
    'proc_vector_size': 60,
    'iterations': 30,
    'window_size_days': 2 * 365
}

diag_glove_rep, proc_glove_rep = glove.glove_representation(**glove_args)

In [20]:
print(f'#point_indices: {len(subjects_interface.nth_points)}')
print(f'#total_points: {sum(len(points) for n, points in subjects_interface.nth_points.items())}')

#[len(points) for n, points in subjects_interface.nth_points.items()]

#point_indices: 1085
#total_points: 129334


## GRAM objects

In [21]:
from datetime import datetime
daily_tracer = "/tmp/tensorboard/"+ datetime.now().strftime("%Y%m%d-%H%M%S") 
print(daily_tracer)

/tmp/tensorboard/20211205-141308


In [22]:


logs = '/tmp/tensorboard/20210708-182059'
#server = jax.profiler.start_server(9999)

In [23]:
       
config = {
    'gram_config': {
        'diag': {
            'ccs_dag': KG,
            'code2index': subjects_interface.diag_multi_ccs_idx,
            'attention_method': 'tanh', #l2, tanh
            'attention_dim': 50,
            'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
            'basic_embeddings': diag_glove_rep
        },
        'proc': {
            'ccs_dag': KG,
            'code2index': subjects_interface.proc_multi_ccs_idx,
            'attention_method': 'tanh',
            'attention_dim': 50,
            'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
            'basic_embeddings': proc_glove_rep
        }
    },
    'model': {
        'ode_dyn': 'mlp', # gru, mlp
        'state_size': 50,
        'numeric_hidden_size': 50,
        'bias': True
    },
    'training': {
        'train_validation_split': 0.8,
        'batch_size': 4,
        'epochs': 200,
        'lr': 1e-3,
        'diag_loss': 'balanced_focal', # balanced_focal, bce
        'tay_reg': 3, # Order of regularized derivative of the dynamics function (None for disable).
        'loss_mixing': {
            'num_alpha': 0.1,
            'diag_alpha': 0.1,
            'ode_alpha': 1e-3,
            'l1_reg': 1e-6,
            'l2_reg': 1e-5,
            'dyn_reg': 1e-5
        },
        'eval_freq': 10,
        'save_freq': 100,
        'save_params_prefix': None
    }
}


In [24]:
       
# config = {
#     'gram_config': {
#         'diag': {
#             'ccs_dag': KG,
#             'code2index': subjects_interface.diag_multi_ccs_idx,
#             'attention_method': 'tanh', #l2, tanh
#             'attention_dim': 150,
#             'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
#             'basic_embeddings': diag_glove_rep
#         },
#         'proc': {
#             'ccs_dag': KG,
#             'code2index': subjects_interface.proc_multi_ccs_idx,
#             'attention_method': 'tanh',
#             'attention_dim': 100,
#             'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
#             'basic_embeddings': proc_glove_rep
#         }
#     },
#     'model': {
#         'ode_dyn': 'mlp', # gru, mlp
#         'state_size': 120,
#         'numeric_hidden_size': 200,
#         'bias': True
#     },
#     'training': {
#         'train_validation_split': 0.8,
#         'batch_size': 20,
#         'epochs': 200,
#         'lr': 1e-3,
#         'diag_loss': 'balanced_focal', # balanced_focal, bce
#         'tay_reg': 2, # Order of regularized derivative of the dynamics function (None for disable).
#         'loss_mixing': {
#             'num_alpha': 0.1,
#             'diag_alpha': 0.1,
#             'ode_alpha': 1e-3,
#             'l1_reg': 1e-6,
#             'l2_reg': 1e-5,
#             'dyn_reg': 1e-5
#         },
#         'eval_freq': 10,
#         'save_freq': 100,
#         'save_params_prefix': None
#     }
# }


In [25]:
diag_gram = gram.DAGGRAM(**config['gram_config']['diag'])

In [26]:
proc_gram = gram.DAGGRAM(**config['gram_config']['proc'])

## GRU-ODE-Bayes

In [27]:
import random
%load_ext autoreload
%autoreload 2
from absl import logging
logging.set_verbosity(logging.INFO)

#with jax.profiler.trace(logs):
res = ode.train_ehr(subject_interface=subjects_interface,
                diag_gram=diag_gram,
                proc_gram=proc_gram,
                rng=random.Random(42),
                **config['model'],
                **config['training'],
                verbose_debug=False,
                shape_debug=False,
                nan_debug=False,
                memory_profile=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:ode:#params: 343234
INFO:ode:shape(params): {'diag_gram': ((589, 100), FlatMap({
  'None_DAG_Attention/~/linear': FlatMap({'b': (50,), 'w': (200, 50)}),
  'None_DAG_Attention/~/linear_1': FlatMap({'w': (50, 1)}),
})), 'f_dec': FlatMap({
  'f_dec/~/lin_gram': FlatMap({'b': (100,), 'w': (150, 100)}),
  'f_dec/~/lin_h_hidden': FlatMap({'b': (50,), 'w': (50, 50)}),
  'f_dec/~/lin_num_hidden1': FlatMap({'b': (50,), 'w': (550, 50)}),
  'f_dec/~/lin_num_hidden2': FlatMap({'b': (100,), 'w': (50, 100)}),
  'f_dec/~/lin_out': FlatMap({'b': (284,), 'w': (100, 284)}),
}), 'f_num': FlatMap({
  'f_numeric/linear': FlatMap({'b': (50,), 'w': (50, 50)}),
  'f_numeric/~/linear': FlatMap({'b': (550,), 'w': (50, 550)}),
  'f_numeric/~/linear_1': FlatMap({'b': (550,), 'w': (50, 550)}),
}), 'f_state_init': FlatMap({
  'f_init/~/lin_hidden1': FlatMap({'b': (200,), 'w': (110, 200)}),
  'f_init/~/lin_hidden2': FlatMap({'b': (100,), 'w': (200, 100)}),
  'f_init/~/lin_state': FlatMap({'b': (50,), 'w': (100,

KeyError: 'age'

#### Possible modifications:
- Add more layers to the adjustment function
- Use days instead of weeks for odeint